In [2]:
import numpy as np
import pandas as pd
import sqlalchemy as sa
import pudl
import tarfile
import splink
from financial_entity_cleaner.text.name import CompanyNameCleaner

from pudl import logging_helpers
logger = pudl.logging_helpers.get_logger(__name__)

# CorpsWatch SEC EX-21 download
I downloaded the CSV's from this file:
https://archive.org/download/corpwatch_api_data_dumps/corpwatch_api_tables_csv.tar.gz

Unzipped the tarball and put it in a dir in *this* directory.

# SEC EX-21 Extract
I downloaded the CSv's from this file:
https://archive.org/download/corpwatch_api_data_dumps/corpwatch_api_tables_csv.tar.gz

And put it in a dir in *this* directory.

In [2]:
directory = "corpwatch_api_tables_csv"
# tables to grab
tables_sec21 = ["companies", "company_relations", "company_info", "sic_codes", "company_locations"]

for table in tables_sec21:
    logger.info(f"{table}: extracting CSV")
    file = tarfile.open(f"../{directory}/corpwatch_api_tables_csv.tar.gz")
    file.extract(f"corpwatch_api_tables_csv/{table}.csv", f"../{directory}")
    file.close()
    vars()[table] = pd.read_csv(f"../{directory}/corpwatch_api_tables_csv/{table}.csv", sep='\t').convert_dtypes()

2023-03-14 09:30:01 [    INFO] catalystcoop.__main__:6 companies: extracting CSV
2023-03-14 09:30:33 [    INFO] catalystcoop.__main__:6 company_relations: extracting CSV
2023-03-14 09:30:58 [    INFO] catalystcoop.__main__:6 company_info: extracting CSV
/var/folders/cs/pq2nw6_x49l0mx3k8h6lps440000gn/T/ipykernel_16985/3906895137.py:10: DtypeWarning: Columns (8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  vars()[table] = pd.read_csv(f"../{directory}/corpwatch_api_tables_csv/{table}.csv", sep='\t').convert_dtypes()
2023-03-14 09:31:51 [    INFO] catalystcoop.__main__:6 sic_codes: extracting CSV
2023-03-14 09:32:19 [    INFO] catalystcoop.__main__:6 company_locations: extracting CSV
/var/folders/cs/pq2nw6_x49l0mx3k8h6lps440000gn/T/ipykernel_16985/3906895137.py:10: DtypeWarning: Columns (5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  vars()[table] = pd.read_csv(f"../{directory}/corpwatch_api_tables_csv/{table}.csv", 

In [28]:
# there are nulls for cw_id, but otherwise it is unique
companies[companies.duplicated(subset=["cw_id"], keep=False)]

,row_id,cw_id,cik,company_name,source_type,source_id
690443,72172,<NA>,0,Dentrix Dental Systems Inc,relationships,285
690444,72173,<NA>,0,Henry Schein Europe Inc,relationships,286
690445,72174,<NA>,0,Henry Schein Financial Services Inc,relationships,287
690446,72175,<NA>,0,Henry Schein Holding GmbH,relationships,289
690447,72179,<NA>,0,Medallion Bank,relationships,134
...,...,...,...,...,...,...
1326523,1937755,<NA>,<NA>,| | | | + Canada On Deck Asset Funding LP,relationships,4575423
1326524,1937756,<NA>,<NA>,| | | | + CNU of Louisiana LLC,relationships,4394297
1326525,1937757,<NA>,<NA>,| | | | + NC Financial Solutions of Louisiana LLC,relationships,4394298
1326526,1937758,<NA>,<NA>,| | | | + NC Financial Solutions of Utah LLC,relationships,4394299


In [32]:
# doesn't seem like null cw_id's have other company records
companies[companies.company_name.str.contains("Dentrix")]

,row_id,cw_id,cik,company_name,source_type,source_id
690443,72172,<NA>,0,Dentrix Dental Systems Inc,relationships,285


In [19]:
company_relations.head(3)

,relation_id,source_cw_id,target_cw_id,relation_type,relation_origin,origin_id,year
0,1,54894,1,<NA>,relationships,2507504,2010
1,86,22569,37,<NA>,relationships,3086044,2014
2,98,9019,45,<NA>,relationships,3285502,2015


In [20]:
company_info.head(3)

,row_id,cw_id,most_recent,year,cik,irs_number,best_location_id,sic_code,industry_name,sic_sector,sector_name,source_type,source_id,num_parents,num_children,top_parent_id,company_name,max_year,min_year,no_sic
0,1,1,0,2010,20,221759452,1,3823,"Industrial instruments for measurement, displa...",3800,Instruments and related products,filers,579799,1,20,54894,K TRON INTERNATIONAL INC,2011,2003,0
1,2,37,0,2003,3153,630004250,78,4911,Electric services,4900,"Electric, gas, and sanitary services",filers,52734,6,36,37,ALABAMA POWER CO,2022,2003,0
2,3,37,0,2004,3153,630004250,78,4911,Electric services,4900,"Electric, gas, and sanitary services",filers,116021,6,37,37,ALABAMA POWER CO,2022,2003,0


In [34]:
test = company_info.merge(sic_codes, on=["sic_code", "industry_name", "sic_sector"], how="left", indicator=True)

In [42]:
# there are some sic codes in the company info table that aren't in the sic_codes table
test[(test._merge=="left_only") & ~(test.sic_code.isnull())].sic_code.unique()

<IntegerArray>
[0, 1044, 6120]
Length: 3, dtype: Int64

In [44]:
sic_codes[sic_codes.sic_code==6120]

,sic_code,industry_name,sic_sector


In [45]:
company_locations.location_id.is_unique

True

In [50]:
# min_year and max_year represent how long an address is applicable for, there might be a change in address after max_year
company_locations[company_locations.street_1.str.contains("PITMAN ROAD")]

,location_id,cw_id,date,type,raw_address,street_1,street_2,city,state,postal_code,country,country_code,subdiv_code,min_year,max_year,most_recent
199392,199393,388465,2009-10-21,business,"29 PITMAN ROAD, BARRE VT 06542",29 PITMAN ROAD,<NA>,BARRE,VT,06542,<NA>,US,VT,2009,2014,1
320688,320689,801059,2014-04-24,business,"29 PITMAN ROAD, BARRE VT 05641",29 PITMAN ROAD,<NA>,BARRE,VT,05641,<NA>,US,VT,2014,2019,1
848084,848085,388465,2009-10-21,mailing,"29 PITMAN ROAD, BARRE VT 06542",29 PITMAN ROAD,<NA>,BARRE,VT,06542,<NA>,US,VT,2009,2014,1
1011864,1011865,801059,2014-04-24,mailing,"29 PITMAN ROAD, BARRE VT 05641",29 PITMAN ROAD,<NA>,BARRE,VT,05641,<NA>,US,VT,2014,2019,1


In [58]:
# it's not entirely clear what this date column represents in the company locations, the date that the record was entered?
loc_test = (
        company_locations.dropna(subset=["cw_id"])
        .assign(loc_year=lambda x: pd.to_datetime(company_locations.date).dt.year).convert_dtypes()
    )

In [86]:
loc_test[loc_test.min_year != loc_test.loc_year].head(3)

,location_id,cw_id,date,type,raw_address,street_1,street_2,city,state,postal_code,country,country_code,subdiv_code,min_year,max_year,most_recent,loc_year
120,121,57,2004-02-27,business,"950 EAST BASSE ROAD, SECOND FLOOR, SAN ANTONIO...",950 EAST BASSE ROAD,SECOND FLOOR,SAN ANTONIO,TX,78209-1831,<NA>,US,TX,2003,2022,1,2004
138,139,66,2004-10-28,business,"333 S HOPE ST - 52ND FLOOR, LOS ANGELES CA 90071",333 S HOPE ST - 52ND FLOOR,<NA>,LOS ANGELES,CA,90071,<NA>,US,CA,2003,2004,1,2004
161,162,77,2004-11-24,business,"ONE CHRISTINA CENTRE 301 N WALNUT STREET, SUIT...",ONE CHRISTINA CENTRE 301 N WALNUT STREET,SUITE 1002,WILMINGTON,DE,19801-2919,<NA>,US,DE,2003,2018,1,2004


In [56]:
# there will be a new location id for the same company when the location changes
# there will also be a different record for each location type
loc_test[loc_test.cw_id == 1]

,location_id,cw_id,date,type,raw_address,street_1,street_2,city,state,postal_code,country,country_code,subdiv_code,min_year,max_year,most_recent,year
0,1,1,2003-10-21,business,"ROUTE 55 & 553, BOX 888, PITMAN NJ 08071-0888",ROUTE 55 & 553,BOX 888,PITMAN,NJ,08071-0888,<NA>,US,NJ,2003,2011,1,2003
586944,586945,1,2003-10-21,mailing,"ROUTE 55 & 553, P O BOX 888, PITMAN NJ 08071-0888",ROUTE 55 & 553,P O BOX 888,PITMAN,NJ,08071-0888,<NA>,US,NJ,2003,2011,1,2003
1345946,1345947,1,2003-10-21,state_of_incorp,NJ,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,US,NJ,2003,2011,1,2003


In [57]:
loc_test[loc_test.location_id == 1]

,location_id,cw_id,date,type,raw_address,street_1,street_2,city,state,postal_code,country,country_code,subdiv_code,min_year,max_year,most_recent,year
0,1,1,2003-10-21,business,"ROUTE 55 & 553, BOX 888, PITMAN NJ 08071-0888",ROUTE 55 & 553,BOX 888,PITMAN,NJ,08071-0888,<NA>,US,NJ,2003,2011,1,2003


In [63]:
# try merging on best_location_id
loc_id_only = company_info.merge(loc_test, how="left", left_on="best_location_id", right_on="location_id", indicator=True)

In [72]:
# there's one location id that isn't in the locations df
loc_id_only[loc_id_only._merge == "left_only"].best_location_id.unique()

<IntegerArray>
[2342732]
Length: 1, dtype: Int64

In [76]:
# seems like best_location_id locations correspond to the correct companies
loc_id_only[loc_id_only.cw_id_x != loc_id_only.cw_id_y]

,row_id,cw_id_x,most_recent_x,year,cik,irs_number,best_location_id,sic_code,industry_name,sic_sector,sector_name,source_type,source_id,num_parents,num_children,top_parent_id,company_name,max_year_x,min_year_x,no_sic,location_id,cw_id_y,date,type,raw_address,street_1,street_2,city,state,postal_code,country,country_code,subdiv_code,min_year_y,max_year_y,most_recent_y,loc_year,_merge


In [80]:
# so potentially some of these best_location_id's are wrong, are there better location_id's to use?
loc_id_only[(loc_id_only.year > loc_id_only.max_year_y) | (loc_id_only.year < loc_id_only.min_year_y)].head(3)

,row_id,cw_id_x,most_recent_x,year,cik,irs_number,best_location_id,sic_code,industry_name,sic_sector,sector_name,source_type,source_id,num_parents,num_children,top_parent_id,company_name,max_year_x,min_year_x,no_sic,location_id,cw_id_y,date,type,raw_address,street_1,street_2,city,state,postal_code,country,country_code,subdiv_code,min_year_y,max_year_y,most_recent_y,loc_year,_merge
1764250,1764251,635924,1,2022,1517342,0,917707,1381,Drilling oil & gas wells,1300,Oil and gas extraction,filers,1838476,2,0,23277,PACIFIC DRILLING SA,2022,2011,0,917707,635924,2011-04-18,mailing,"16 AVENUE PASTEUR, L-2310, NONE N4 NONE",16 AVENUE PASTEUR,L-2310,NONE,N4,NONE,<NA>,LU,<NA>,2011,2014,1,2011,both
2859579,2859580,550,0,2003,30088,<NA>,1158,<NA>,<NA>,<NA>,<NA>,filers,59355,0,0,550,DRESDNER BANK AG,2009,2003,0,1158,550,2004-11-16,business,"GALLUSANLAGE 7-8, 6000 FRANKFURT MAIN, WEST GE...",GALLUSANLAGE 7-8,6000 FRANKFURT MAIN,WEST GERMANY,2M,00000,<NA>,DE,<NA>,2004,2009,1,2004,both
2915443,2915444,4393,0,2018,766011,0,9075,1040,Gold and silver ores,1000,Metal mining,filers,1336181,0,0,4393,CALEDONIA MINING CORP,2023,2003,0,9075,4393,2003-06-16,business,"9-2145 DUNNIN DR, MISSISSAUGA ONTARIO, CANADA ...",9-2145 DUNNIN DR,MISSISSAUGA ONTARIO,CANADA,A6,L5L4L9,<NA>,CA,ON,2003,2003,1,2003,both


In [84]:
# a very small percentage have years that don't fall within the correct range
len(loc_id_only[(loc_id_only.year > loc_id_only.max_year_y) | (loc_id_only.year < loc_id_only.min_year_y)])/len(loc_id_only)

0.00020087518393100995

In [82]:
# for example this company changed locations in 2015, so the location id in the company info should have been updated
company_locations[company_locations.cw_id == 635924]

,location_id,cw_id,date,type,raw_address,street_1,street_2,city,state,postal_code,country,country_code,subdiv_code,min_year,max_year,most_recent
250077,250078,635924,2011-04-18,business,"16 AVENUE PASTEUR, L-2310, NONE N4 NONE, NONE",16 AVENUE PASTEUR,<NA>,L-2310,<NA>,<NA>,<NA>,<NA>,<NA>,2011,2014,1
250078,250079,635924,2015-01-09,business,"8-10 AVENUE DE LA GARE, L-1610, NONE N4 NONE, ...",8-10 AVENUE DE LA GARE,<NA>,L-1610,<NA>,<NA>,<NA>,<NA>,<NA>,2015,2022,1
917706,917707,635924,2011-04-18,mailing,"16 AVENUE PASTEUR, L-2310, NONE N4 NONE",16 AVENUE PASTEUR,L-2310,NONE,N4,NONE,<NA>,LU,<NA>,2011,2014,1
917707,917708,635924,2015-01-09,mailing,"8-10 AVENUE DE LA GARE, L-1610, NONE N4 NONE",8-10 AVENUE DE LA GARE,L-1610,NONE,N4,NONE,<NA>,LU,<NA>,2015,2022,1
1477764,1477765,635924,2011-04-18,state_of_incorp,N4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,LU,<NA>,2011,2022,1


Summary:
- `best_location_id` is mostly reliable, .02% of records have a `best_location_id` where the the location changed and it's using the previous location
- this could be fixed by taking these bad records and getting the record for that `cw_id` and `type` from the `company_location` dataframe that has a `min_year` and `max_year` that contains the `year` for the record
- for now it seems fine to use `best_location_id` instead of using `cw_id`, `year`, and `type` as the join key
- this `max_year`, `min_year`, `year` issue doesn't seem to be an issue for electricity sector companies

In [89]:
elec = company_info[company_info.sic_sector.isin([4900])]

In [91]:
elec_loc = elec.merge(loc_test, how="left", left_on="best_location_id", right_on="location_id")

In [92]:
len(elec_loc[(elec_loc.year > elec_loc.max_year_y) | (elec_loc.year < elec_loc.min_year_y)])/len(elec_loc)

0.0

# Input Prep

In [3]:
idx_eia = ["utility_id_eia", "report_date"]
# when you use best_location as the merge key you no longer need type in the key
idx_sec = ["cw_id", "loc_year", "type"]
shared_cols = ["year", "utility_name_eia", "city", "street_address", "address_2", "zip_code"]

## `pudl_out` setup

In [80]:
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings['pudl_db'])

pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine=pudl_engine)
utils_eia860 = pudl_out.utils_eia860().assign(year=lambda x: x.report_date.dt.year)

In [81]:
# add on ownership_eia860 table here? ignore for now because street address is null
own_eia860 = pudl_out.own_eia860().assign(year=lambda x: x.report_date.dt.year)

2023-03-14 12:04:36 [    INFO] catalystcoop.pudl.transform.eia861:456 Started with 81263 missing BA Codes out of 185551 records (43.80%)
2023-03-14 12:04:41 [    INFO] catalystcoop.pudl.transform.eia861:480 Ended with 14921 missing BA Codes out of 185551 records (8.04%)
2023-03-14 12:04:42 [    INFO] catalystcoop.pudl.output.eia860:177 91.7% of plant records have consistently reported BA Codes
2023-03-14 12:04:42 [    INFO] catalystcoop.pudl.output.eia860:227 Before any filling treatment has been applied. 43.8% of records have no BA codes
2023-03-14 12:04:42 [    INFO] catalystcoop.pudl.output.eia860:227 Backfilling and consistent value is the same. Filled w/ most consistent BA code. 10.8% of records have no BA codes
2023-03-14 12:04:42 [    INFO] catalystcoop.pudl.output.eia860:227 SWPP is most consistent value. Filled w/ oldest BA code. 8.3% of records have no BA codes
2023-03-14 12:04:42 [    INFO] catalystcoop.pudl.output.eia860:227 NWMT is most consistent value. Filled w/ oldest B

In [88]:
own_util_ids = own_eia860[~own_eia860.owner_utility_id_eia.isin(utils_eia860.utility_id_eia.unique())].owner_utility_id_eia.unique()

In [92]:
own_utils = own_eia860[~(own_eia860.owner_name.isnull()) & (own_eia860.owner_utility_id_eia.isin(own_util_ids))]

In [93]:
own_utils

,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,owner_utility_id_eia,owner_name,data_maturity,fraction_owned,owner_city,owner_country,owner_state,owner_street_address,owner_zip_code,year
34227,2010-01-01,57372,5650,Bellevue Solar Project,<NA>,1591,EnXco,INV-1,56714,enXco Development Corporation,final,1.0000,<NA>,USA,CA,<NA>,<NA>,2010
34231,2010-01-01,57374,5652,Lakefield Wind Project LLC,<NA>,1591,EnXco,LAKE1,56714,enXco Development Corporation,final,1.0000,<NA>,USA,CA,<NA>,<NA>,2010
34863,2011-01-01,3118,1093,Conemaugh,15873,1870,RRI Energy NE Management Co,1,56818,Conemaugh Power LLC,final,0.0372,<NA>,USA,NJ,<NA>,<NA>,2011
34970,2011-01-01,3452,2318,Lake Hubbard,55983,2328,Luminant Generation Company LLC,1,57096,Luminant Gen Co LLC Fin Holding,final,1.0000,<NA>,<NA>,<NA>,<NA>,<NA>,2011
34971,2011-01-01,3452,2318,Lake Hubbard,55983,2328,Luminant Generation Company LLC,2,57096,Luminant Gen Co LLC Fin Holding,final,1.0000,<NA>,<NA>,<NA>,<NA>,<NA>,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84435,2021-01-01,65945,17184,Brookville Smart Bus Depot Microgrid,65118,14205,AlphaStruxure Service Co LP,PV,65119,AlphaStruxure Montgomery County LLC,final,1.0000,Boston,USA,MA,"201 Washington St, Floor 27",02111,2021
84436,2021-01-01,65958,17197,WMATA - Anacostia,61944,6113,GSRP,GEN1,65140,"Solar Star Track Anacostia, LLC",final,1.0000,Boca Raton,USA,FL,750 Park of Commerce Blvd,33487,2021
84437,2021-01-01,65959,17198,WMATA - Cheverly Metro,61944,6113,GSRP,GEN1,65141,"Solar Star Track Cheverly, LLC",final,1.0000,Boca Raton,USA,FL,750 Park of Commerce Blvd,33487,2021
84438,2021-01-01,65960,17199,WMATA - Naylor Rd. Metro,61944,6113,GSRP,GEN1,65142,"Solar Star Track, LLC",final,1.0000,Boca Raton,USA,FL,750 Park of Commerce Blvd,33487,2021


In [94]:
own_cols = {"owner_name": "utility_name_eia",
            "owner_city": "city",
            "owner_street_address": "street_address",
            "owner_zip_code": "zip_code",
            "year": "year"}
own_utils = own_utils[list(own_cols.keys())].rename(columns=own_cols)

In [95]:
own_utils

,utility_name_eia,city,street_address,zip_code,year
34227,enXco Development Corporation,<NA>,<NA>,<NA>,2010
34231,enXco Development Corporation,<NA>,<NA>,<NA>,2010
34863,Conemaugh Power LLC,<NA>,<NA>,<NA>,2011
34970,Luminant Gen Co LLC Fin Holding,<NA>,<NA>,<NA>,2011
34971,Luminant Gen Co LLC Fin Holding,<NA>,<NA>,<NA>,2011
...,...,...,...,...,...
84435,AlphaStruxure Montgomery County LLC,Boston,"201 Washington St, Floor 27",02111,2021
84436,"Solar Star Track Anacostia, LLC",Boca Raton,750 Park of Commerce Blvd,33487,2021
84437,"Solar Star Track Cheverly, LLC",Boca Raton,750 Park of Commerce Blvd,33487,2021
84438,"Solar Star Track, LLC",Boca Raton,750 Park of Commerce Blvd,33487,2021


In [104]:
utils_eia860 = pd.concat([utils_eia860, own_utils])

## SEC setup

In [6]:
def prep_companies_sec(company_info, company_locations):
    # either grab both sectors that have 
    sic_sector_electric = (
        # company_info.loc[company_info.sector_name.str.lower().str.contains("electric"), "sic_sector"].unique()
        [4900]
    )
    # maybe we don't need this comapny year anymore
    company_locations = (
        company_locations.dropna(subset=["cw_id"])
        .assign(loc_year=lambda x: pd.to_datetime(company_locations.date).dt.year).convert_dtypes()
    )

    dupe_address_ratio = (
        len(company_locations[company_locations.duplicated(subset=idx_sec, keep=False)])
        /len(company_locations)
    )
    if dupe_address_ratio > .0001:
        raise AssertionError(
            f"{dupe_address_ratio:.3%} of addresses are duplicates based on {idx_sec}. "
            "More than expected (.01%)."
        )
    companies_sec = (
        company_info
        [company_info.sic_sector.isin(sic_sector_electric)]
        .merge(
            company_locations,
            left_on="best_location_id",
            right_on="location_id",
            validate="m:1",
            how="left"
        )
        .rename(columns={
            "company_name": "utility_name_eia",
            "city": "city",
            "street_1": "street_address",
            "street_2": "address_2",
            "postal_code": "zip_code"
        })
    )
    return companies_sec

In [7]:
companies_sec = prep_companies_sec(company_info, company_locations)

In [8]:
companies_sec.head(3)

,row_id,cw_id_x,most_recent_x,year,cik,irs_number,best_location_id,sic_code,industry_name,sic_sector,sector_name,source_type,source_id,num_parents,num_children,top_parent_id,utility_name_eia,max_year_x,min_year_x,no_sic,location_id,cw_id_y,date,type,raw_address,street_address,address_2,city,state,zip_code,country,country_code,subdiv_code,min_year_y,max_year_y,most_recent_y,loc_year
0,2,37,0,2003,3153,630004250,78,4911,Electric services,4900,"Electric, gas, and sanitary services",filers,52734,6,36,37,ALABAMA POWER CO,2022,2003,0,78,37,2003-11-24,business,"600 N 18TH ST, P O BOX 2641, BIRMINGHAM AL 35291",600 N 18TH ST,P O BOX 2641,BIRMINGHAM,AL,35291,<NA>,US,AL,2003,2022,1,2003
1,3,37,0,2004,3153,630004250,78,4911,Electric services,4900,"Electric, gas, and sanitary services",filers,116021,6,37,37,ALABAMA POWER CO,2022,2003,0,78,37,2003-11-24,business,"600 N 18TH ST, P O BOX 2641, BIRMINGHAM AL 35291",600 N 18TH ST,P O BOX 2641,BIRMINGHAM,AL,35291,<NA>,US,AL,2003,2022,1,2003
2,4,37,0,2005,3153,630004250,78,4911,Electric services,4900,"Electric, gas, and sanitary services",filers,212295,6,36,37,ALABAMA POWER CO,2022,2003,0,78,37,2003-11-24,business,"600 N 18TH ST, P O BOX 2641, BIRMINGHAM AL 35291",600 N 18TH ST,P O BOX 2641,BIRMINGHAM,AL,35291,<NA>,US,AL,2003,2022,1,2003


In [9]:
utils_eia860[shared_cols]

,year,utility_name_eia,city,street_address,address_2,zip_code
0,2022,A E Staley Manufacturing Co,<NA>,<NA>,<NA>,<NA>
1,2021,A E Staley Manufacturing Co,Decatur,2200 East Eldorado Street,<NA>,62525
2,2020,A E Staley Manufacturing Co,Decatur,2200 East Eldorado Street,<NA>,62525
3,2019,A E Staley Manufacturing Co,Decatur,2200 East Eldorado Street,<NA>,62525
4,2018,A E Staley Manufacturing Co,Decatur,2200 East Eldorado Street,<NA>,62525
...,...,...,...,...,...,...
119360,2007,Other,-,-,-,<NA>
119361,2006,Other,-,-,-,<NA>
119362,2005,Other,-,-,-,<NA>
119363,2004,Other,-,-,-,<NA>


In [10]:
companies_sec[shared_cols]

,year,utility_name_eia,city,street_address,address_2,zip_code
0,2003,ALABAMA POWER CO,BIRMINGHAM,600 N 18TH ST,P O BOX 2641,35291
1,2004,ALABAMA POWER CO,BIRMINGHAM,600 N 18TH ST,P O BOX 2641,35291
2,2005,ALABAMA POWER CO,BIRMINGHAM,600 N 18TH ST,P O BOX 2641,35291
3,2006,ALABAMA POWER CO,BIRMINGHAM,600 N 18TH ST,P O BOX 2641,35291
4,2007,ALABAMA POWER CO,BIRMINGHAM,600 N 18TH ST,P O BOX 2641,35291
...,...,...,...,...,...,...
9289,2023,Excelerate Energy Holdings LLC,TULSA,6733 S. YALE AVE.,<NA>,74136
9290,2023,Ambipar Emergency Response,SAO PAULO,"AVENIDA ANGELICA, N. 2346","5TH FLOOR, ROOM 4, CONSOLACAO",01228-200
9291,2022,MN8 Energy Inc,NEW YORK,200 WEST STREET,<NA>,10282
9292,2023,MN8 Energy Inc,NEW YORK,200 WEST STREET,<NA>,10282


# Cleaning

In [128]:
len(companies_sec[companies_sec.street_address.isnull()])/len(companies_sec)

0.0018291370776845276

In [135]:
utils_eia860.loc[(utils_eia860.street_address.isnull()) & ~(utils_eia860.address_2.isnull()), "street_address"] = utils_eia860.address_2

In [136]:
len(utils_eia860[utils_eia860.street_address.isnull()])/len(utils_eia860)

0.15675449252293386

In [137]:
null_util = utils_eia860[utils_eia860.street_address.isnull()][shared_cols]

In [140]:
null_util.city.isnull().value_counts()

True     13943
False     4768
Name: city, dtype: int64

Other cleaning steps that should be done:
- replace abbreviations with full names - st. -> street

In [36]:
# can also coopt as an address cleaner
company_cleaner = CompanyNameCleaner()
company_cleaner.default_cleaning_rules

['replace_amperstand_between_space_by_AND',
 'replace_hyphen_between_spaces_by_single_space',
 'replace_underscore_between_spaces_by_single_space',
 'remove_text_puctuation_except_dot',
 'remove_math_symbols',
 'remove_words_in_parentheses',
 'remove_parentheses',
 'remove_brackets',
 'remove_curly_brackets',
 'enforce_single_space_between_words']

In [ ]:
address_cleaner = CompanyNameCleaner()
address_cleaner.default_cleaning_rules.append("remove_all_punctuation")

In [114]:
# is there a way to fix the ltd to become limited?
company_cleaner.get_clean_data("ltd co")

'ltd company'

In [204]:
# adapted from https://stackoverflow.com/questions/58907196/data-cleaning-addresses-python
def lengthen_address(df, cols):
    df = df.copy()
    replace_list = [r" st(?=$| [ne(so|s$)(we|w$)])",
                    r" rd(?=$| [ne(so|s$)(we|w$)])",
                    r" ave(?=$| [ne(so|s$)(we|w$)])",
                    r" cl(?=$| [ne(so|s$)(we|w$)])",
                    r" ct(?=$| [ne(so|s$)(we|w$)])",
                    r" cres(?=$| [ne(so|s$)(we|w$)])",
                    r" blvd(?=$| [ne(so|s$)(we|w$)])",
                    r" dr(?=$| [ne(so|s$)(we|w$)])",
                    r" ln(?=$| [ne(so|s$)(we|w$)])",
                    r" pl(?=$| [ne(so|s$)(we|w$)])",
                    r" sq(?=$| [ne(so|s$)(we|w$)])",
                    r" pde(?=$| [ne(so|s$)(we|w$)])",
                    r" cct(?=$| [ne(so|s$)(we|w$)])",
                    r" ste ",
                    r" n ",
                    r" s ",
                    r" e ",
                    r" w ",
                    r" ne ",
                    r" nw ",
                    r" se ",
                    r" sw ",]
    
    value_list = [" street", " road", " avenue", " close", " court", " crescent", " boulevard", " drive", " lane", " place", " square", " parade", " circuit", " suite",
                 " north ", " south ", " east ", " west ", " northeast ", " northwest ", " southeast ", " southwest "]
    df.loc[:, cols] = df[cols].replace(to_replace=replace_list, value=value_list, regex=True)
    return df

In [158]:
test = sec_df.head(6)

In [180]:
test

,year,utility_name_eia,city,street_address,address_2,zip_code,utility_name_clean,street_address_clean,address_2_clean,unique_id
0,2003,alabama power co,birmingham,600 north 18th street,p o box 2641,35291,alabama power company,600 n 18th st,p o box 2641,0
1,2004,alabama power co,birmingham,600 north 18th street,p o box 2641,35291,alabama power company,600 n 18th st,p o box 2641,1
2,2005,alabama power co,birmingham,600 north 18th street,p o box 2641,35291,alabama power company,600 n 18th st,p o box 2641,2
3,2006,alabama power co,birmingham,600 north 18th street,p o box 2641,35291,alabama power company,600 n 18th st,p o box 2641,3
4,2007,alabama power co,birmingham,600 north 18th street,p o box 2641,35291,alabama power company,600 n 18th st,p o box 2641,4
5,2008,alabama power co,birmingham,600 n 18th st,p o box 2641,35291,alabama power company,600 n 18th st,p o box 2641,5


In [181]:
lengthen_address(test, ["street_address", "address_2_clean"])

,year,utility_name_eia,city,street_address,address_2,zip_code,utility_name_clean,street_address_clean,address_2_clean,unique_id
0,2003,alabama power co,birmingham,600 north 18th street,p o box 2641,35291,alabama power company,600 n 18th st,p o box 2641,0
1,2004,alabama power co,birmingham,600 north 18th street,p o box 2641,35291,alabama power company,600 n 18th st,p o box 2641,1
2,2005,alabama power co,birmingham,600 north 18th street,p o box 2641,35291,alabama power company,600 n 18th st,p o box 2641,2
3,2006,alabama power co,birmingham,600 north 18th street,p o box 2641,35291,alabama power company,600 n 18th st,p o box 2641,3
4,2007,alabama power co,birmingham,600 north 18th street,p o box 2641,35291,alabama power company,600 n 18th st,p o box 2641,4
5,2008,alabama power co,birmingham,600 north 18th street,p o box 2641,35291,alabama power company,600 n 18th st,p o box 2641,5


In [205]:
def clean_df(df):
    df = df[shared_cols]
    types = {"utility_name_eia": str, "city": str, "street_address": str, "address_2": str, "zip_code": str, "year": int}
    df = df.astype(types)
    str_cols = ["utility_name_eia", "city", "street_address", "address_2"]
    for col in str_cols:
        df[col] = df[col].str.strip().str.lower()
    # fix nans with n number of x's
    df = df.replace(["-", ".", "<na>", "<NA>", "nan", "NaN", "x", "xxx", "xxxxxxx"], np.nan)
    # clean company names
    company_cleaner = CompanyNameCleaner()
    df = company_cleaner.get_clean_df(df, "utility_name_eia", "utility_name_clean")
    df.loc[~(df.zip_code.isnull()), "zip_code"] = df["zip_code"].str.zfill(5)
    # replace null addresses with address_2 if it's not null
    # is this a good idea?
    df.loc[(df.street_address.isnull()) & ~(df.address_2.isnull()), "street_address"] = df.address_2
    # coopt company name cleaner for addresses
    address_cleaner = CompanyNameCleaner()
    address_cleaner.default_cleaning_rules.append("remove_all_punctuation")
    df = address_cleaner.get_clean_df(df, "street_address", "street_address_clean")
    df = address_cleaner.get_clean_df(df, "address_2", "address_2_clean")
    df = lengthen_address(df, ["street_address_clean", "address_2_clean"])
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    # add unique_id column for splink
    df["unique_id"] = df.index
    return df

In [206]:
utils_eia860 = utils_eia860.reset_index(drop=True)

In [207]:
sec_df = clean_df(companies_sec)
eia_df = clean_df(utils_eia860)

In [208]:
sec_df

,year,utility_name_eia,city,street_address,address_2,zip_code,utility_name_clean,street_address_clean,address_2_clean,unique_id
0,2003,alabama power co,birmingham,600 n 18th st,p o box 2641,35291,alabama power company,600 north 18th street,p o box 2641,0
1,2004,alabama power co,birmingham,600 n 18th st,p o box 2641,35291,alabama power company,600 north 18th street,p o box 2641,1
2,2005,alabama power co,birmingham,600 n 18th st,p o box 2641,35291,alabama power company,600 north 18th street,p o box 2641,2
3,2006,alabama power co,birmingham,600 n 18th st,p o box 2641,35291,alabama power company,600 north 18th street,p o box 2641,3
4,2007,alabama power co,birmingham,600 n 18th st,p o box 2641,35291,alabama power company,600 north 18th street,p o box 2641,4
...,...,...,...,...,...,...,...,...,...,...
9282,2023,excelerate energy holdings llc,tulsa,6733 s. yale ave.,NaN,74136,excelerate energy holdings limited liability c...,6733 south yale avenue,NaN,9282
9283,2023,ambipar emergency response,sao paulo,"avenida angelica, n. 2346","5th floor, room 4, consolacao",01228-200,ambipar emergency response,avenida angelica north 2346,5th floor room 4 consolacao,9283
9284,2022,mn8 energy inc,new york,200 west street,NaN,10282,mn8 energy incorporated,200 west street,NaN,9284
9285,2023,mn8 energy inc,new york,200 west street,NaN,10282,mn8 energy incorporated,200 west street,NaN,9285


In [209]:
eia_df

,year,utility_name_eia,city,street_address,address_2,zip_code,utility_name_clean,street_address_clean,address_2_clean,unique_id
0,2022,a e staley manufacturing co,NaN,NaN,NaN,NaN,a e staley manufacturing company,NaN,NaN,0
1,2021,a e staley manufacturing co,decatur,2200 east eldorado street,NaN,62525,a e staley manufacturing company,2200 east eldorado street,NaN,1
2,2020,a e staley manufacturing co,decatur,2200 east eldorado street,NaN,62525,a e staley manufacturing company,2200 east eldorado street,NaN,2
3,2019,a e staley manufacturing co,decatur,2200 east eldorado street,NaN,62525,a e staley manufacturing company,2200 east eldorado street,NaN,3
4,2018,a e staley manufacturing co,decatur,2200 east eldorado street,NaN,62525,a e staley manufacturing company,2200 east eldorado street,NaN,4
...,...,...,...,...,...,...,...,...,...,...
124336,2021,alphastruxure montgomery county llc,boston,"201 washington st, floor 27",NaN,02111,alphastruxure montgomery county limited liabil...,201 washington st floor 27,NaN,124336
124337,2021,"solar star track anacostia, llc",boca raton,750 park of commerce blvd,NaN,33487,solar star track anacostia limited liability c...,750 park of commerce boulevard,NaN,124337
124338,2021,"solar star track cheverly, llc",boca raton,750 park of commerce blvd,NaN,33487,solar star track cheverly limited liability co...,750 park of commerce boulevard,NaN,124338
124339,2021,"solar star track, llc",boca raton,750 park of commerce blvd,NaN,33487,solar star track limited liability company,750 park of commerce boulevard,NaN,124339


In [186]:
# regex is missing address conversions like this, fix it
eia_df.iloc[124336]

year                                                                 2021
utility_name_eia                      alphastruxure montgomery county llc
city                                                               boston
street_address                                201 washington st, floor 27
address_2                                                             NaN
zip_code                                                            02111
utility_name_clean      alphastruxure montgomery county limited liabil...
street_address_clean                           201 washington st floor 27
address_2_clean                                                       NaN
unique_id                                                          124336
Name: 124336, dtype: object

In [53]:
sec_df.utility_name_clean.sort_values()

7552          3ci complete compliance corporation
7551          3ci complete compliance corporation
7549          3ci complete compliance corporation
7550          3ci complete compliance corporation
8668                  4g enterprises incorporated
                          ...                    
7697     zond windsystem partners ltd series 85 b
7699     zond windsystem partners ltd series 85 c
7700     zond windsystem partners ltd series 85 c
7675                 zurn industries incorporated
7676    zurn industries limited liability company
Name: utility_name_clean, Length: 9287, dtype: object

In [56]:
eia_df.utility_name_clean.sort_values()

118026      0ham wham8 solar limited liability company
118024      0ham wham8 solar limited liability company
118025      0ham wham8 solar limited liability company
114122    10 briggs solar ng limited liability company
114123    10 briggs solar ng limited liability company
                              ...                     
119086                                             NaN
119087                                             NaN
119088                                             NaN
119089                                             NaN
119090                                             NaN
Name: utility_name_clean, Length: 119112, dtype: object

In [215]:
col_rename = {"utility_name_eia": "utility_name_eia_dirty", "utility_name_clean": "utility_name_eia",
              "street_address": "street_address_dirty", "street_address_clean": "street_address",
              "address_2": "address_2_dirty", "address_2_clean": "address_2"}
sec_df = sec_df.rename(columns=col_rename)
eia_df = eia_df.rename(columns=col_rename)

In [219]:
# sort by year in case I want to consider year later on
sec_df = sec_df.sort_values(by="year", ascending=False).drop_duplicates(subset=["utility_name_eia", "street_address", "address_2", "zip_code", "city"])

In [220]:
eia_df = eia_df.sort_values(by="year", ascending=False).drop_duplicates(subset=["utility_name_eia", "street_address", "address_2", "zip_code", "city"])

In [221]:
eia_df

,year,utility_name_eia_dirty,city,street_address_dirty,address_2_dirty,zip_code,utility_name_eia,street_address,address_2,unique_id
0,2022,a e staley manufacturing co,NaN,NaN,NaN,NaN,a e staley manufacturing company,NaN,NaN,0
100170,2022,alpaca energy llc,NaN,NaN,NaN,NaN,alpaca energy limited liability company,NaN,NaN,100170
100266,2022,fluence,NaN,NaN,NaN,NaN,fluence,NaN,NaN,100266
100256,2022,maine township high school dist 207,NaN,NaN,NaN,NaN,maine township high school dist 207,NaN,NaN,100256
100248,2022,port arthur steam energy lp,NaN,NaN,NaN,NaN,port arthur steam energy limited partnership,NaN,NaN,100248
...,...,...,...,...,...,...,...,...,...,...
45478,2001,oxy vinyls lp,dallas,5005 lbj highway,p o box 809050,75380,oxy vinyls limited partnership,5005 lbj highway,p o box 809050,45478
32368,2001,lco hydro,hayward,route 2,p o box 2700,54843,lco hydro,route 2,p o box 2700,32368
68259,2001,washington state university,pullman,dept of facilities of oper,NaN,99164,washington state university,dept of facilities of oper,NaN,68259
8557,2001,calpine corp,gilroy,1400 pacheco pass highway,NaN,95020,calpine corporation,1400 pacheco pass highway,NaN,8557


# Splink Exploratory Analysis

In [200]:
from splink.duckdb.duckdb_linker import DuckDBLinker

In [222]:
# build up settings dict more later
settings_dict = {"link_type": "link_only"}
linker = DuckDBLinker([sec_df, eia_df], settings_dict=settings_dict)

In [223]:
linker.missingness_chart()

Skew in city column.

Name and address are likely the best linking columns, as is expected. Low value counts for each of the values.

In [224]:
cols = ["utility_name_eia", "city", "street_address", "address_2", "zip_code"]
linker.profile_columns(cols)

### Look at blocking rules

In [221]:
# ignore year for now
blocking_rule_0 = "l.year = r.year"
count = linker.count_num_comparisons_from_blocking_rule(blocking_rule_1)
print(f"Number of comparisons generated by '{blocking_rule_1}': {count:,.0f}")

Number of comparisons generated by 'l.year = r.year': 50,203,940


In [235]:
blocking_rule_1 = "jaccard(l.utility_name_eia, r.utility_name_eia) >= .5"
count = linker.count_num_comparisons_from_blocking_rule(blocking_rule_1)
print(f"Number of comparisons generated by '{blocking_rule_1}': {count:,.0f}")

Number of comparisons generated by 'jaccard(l.utility_name_eia, r.utility_name_eia) >= .5': 49,681,529


In [236]:
blocking_rule_10 = "jaccard(l.utility_name_eia, r.utility_name_eia) >= .9"
count = linker.count_num_comparisons_from_blocking_rule(blocking_rule_10)
print(f"Number of comparisons generated by '{blocking_rule_10}': {count:,.0f}")

Number of comparisons generated by 'jaccard(l.utility_name_eia, r.utility_name_eia) >= .9': 379,029


In [234]:
blocking_rule_2 = "levenshtein(l.zip_code, r.zip_code) <= 1"
count = linker.count_num_comparisons_from_blocking_rule(blocking_rule_2)
print(f"Number of comparisons generated by '{blocking_rule_2}': {count:,.0f}")

Number of comparisons generated by 'levenshtein(l.zip_code, r.zip_code) <= 1': 140,833


In [238]:
blocking_rule_3 = "substr(l.city, 1, 4) = substr(r.city, 1, 4)"
count = linker.count_num_comparisons_from_blocking_rule(blocking_rule_3)
print(f"Number of comparisons generated by '{blocking_rule_3}': {count:,.0f}")

Number of comparisons generated by 'substr(l.city, 1, 4) = substr(r.city, 1, 4)': 418,904


In [239]:
blocking_rule_4 = "substr(l.utility_name_eia, 1, 3) = substr(r.utility_name_eia, 1, 3)"
count = linker.count_num_comparisons_from_blocking_rule(blocking_rule_4)
print(f"Number of comparisons generated by '{blocking_rule_4}': {count:,.0f}")

Number of comparisons generated by 'substr(l.utility_name_eia, 1, 3) = substr(r.utility_name_eia, 1, 3)': 146,671


In [241]:
blocking_rule_5 = "substr(l.street_address, 1, 4) = substr(r.street_address, 1, 4)"
count = linker.count_num_comparisons_from_blocking_rule(blocking_rule_5)
print(f"Number of comparisons generated by '{blocking_rule_5}': {count:,.0f}")

Number of comparisons generated by 'substr(l.street_address, 1, 4) = substr(r.street_address, 1, 4)': 190,971


In [242]:
blocking_rule_6 = "substr(l.address_2, 1, 4) = substr(r.address_2, 1, 4)"
count = linker.count_num_comparisons_from_blocking_rule(blocking_rule_6)
print(f"Number of comparisons generated by '{blocking_rule_6}': {count:,.0f}")

Number of comparisons generated by 'substr(l.address_2, 1, 4) = substr(r.address_2, 1, 4)': 251,062


In [243]:
blocking_rules = [blocking_rule_2, blocking_rule_3, blocking_rule_4, blocking_rule_5, blocking_rule_6]
linker.cumulative_num_comparisons_from_blocking_rules_chart(blocking_rules)

TODO: Fix this later

In [292]:
eia_df_2005.iloc[7331].zip_code

'00nan'

# Link datasets

In [257]:
import splink.duckdb.duckdb_comparison_library as cl

utility_name_comparison = cl.jaccard_at_thresholds("utility_name_eia")
print(utility_name_comparison.human_readable_description)

city_name_comparison = cl.levenshtein_at_thresholds("city", [1, 3], term_frequency_adjustments=True)
print(city_name_comparison.human_readable_description)

print(cl.jaccard_at_thresholds("street_address").human_readable_description)

print(cl.jaccard_at_thresholds("address_2").human_readable_description)
print(cl.exact_match("zip_code").human_readable_description)

Comparison 'Exact match vs. jaccard at thresholds 0.9, 0.7 vs. anything else' of "utility_name_eia".
Similarity is assessed using the following ComparisonLevels:
    - 'Null' with SQL rule: "utility_name_eia_l" IS NULL OR "utility_name_eia_r" IS NULL
    - 'Exact match' with SQL rule: "utility_name_eia_l" = "utility_name_eia_r"
    - 'jaccard >= 0.9' with SQL rule: jaccard("utility_name_eia_l", "utility_name_eia_r") >= 0.9
    - 'jaccard >= 0.7' with SQL rule: jaccard("utility_name_eia_l", "utility_name_eia_r") >= 0.7
    - 'All other comparisons' with SQL rule: ELSE

Comparison 'Exact match vs. levenshtein at thresholds 1, 3 vs. anything else' of "city".
Similarity is assessed using the following ComparisonLevels:
    - 'Null' with SQL rule: "city_l" IS NULL OR "city_r" IS NULL
    - 'Exact match' with SQL rule: "city_l" = "city_r"
    - 'levenshtein <= 1' with SQL rule: levenshtein("city_l", "city_r") <= 1
    - 'levenshtein <= 3' with SQL rule: levenshtein("city_l", "city_r") <= 3
 

In [258]:
settings = {
    "link_type": "link_only",
    "comparisons": [
        utility_name_comparison,
        city_name_comparison,
        cl.jaccard_at_thresholds("street_address"),
        # address_2 blocking rule had to be levenshtein, jaccard threw error string was "too short"
        cl.levenshtein_at_thresholds("address_2"),
        cl.exact_match("zip_code"),
    ],
    "blocking_rules_to_generate_predictions": blocking_rules,
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
}

In [259]:
linker = DuckDBLinker([sec_df, eia_df], settings_dict=settings)

Get the probability two random records match

In [263]:
deterministic_rules = [
    "jaccard(l.utility_name_eia, r.utility_name_eia) >= .9 and jaccard(l.street_address, r.street_address) >= .8",
    "jaccard(l.utility_name_eia, r.utility_name_eia) >= .9 and levenshtein(r.city, l.city) <= 1",
    "jaccard(l.utility_name_eia, r.utility_name_eia) >= .9 and levenshtein(l.zip_code, r.zip_code) <= 1",
    "l.street_address = r.street_address"
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.75)


Probability two random records match is estimated to be  0.000249.
This means that amongst all possible pairwise record comparisons, one in 4,008.58 are expected to match.  With 60,951,792 total possible comparisons, we expect a total of around 15,205.33 matching pairs


Estimate the u parameter

In [262]:
linker.estimate_u_using_random_sampling(target_rows=1e6)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - utility_name_eia (no m values are trained).
    - city (no m values are trained).
    - street_address (no m values are trained).
    - address_2 (no m values are trained).
    - zip_code (no m values are trained).


Estimate the m parameter

In [264]:
training_blocking_rule = "l.utility_name_eia = r.utility_name_eia and l.city = r.city"
training_session_fname_sname = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.utility_name_eia = r.utility_name_eia and l.city = r.city

Parameter estimates will be made for the following comparison(s):
    - street_address
    - address_2
    - zip_code

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - utility_name_eia
    - city

Iteration 1: Largest change in params was -0.845 in the m_probability of address_2, level `Exact match`
Iteration 2: Largest change in params was 0.0484 in the m_probability of address_2, level `All other comparisons`
Iteration 3: Largest change in params was -0.00521 in probability_two_random_records_match
Iteration 4: Largest change in params was -0.0047 in probability_two_random_records_match
Iteration 5: Largest change in params was -0.00423 in probability_two_random_records_match
Iteration 6: Largest change in params was -0.00378 in probability_two_random_reco

In [268]:
training_blocking_rule = "l.utility_name_eia = r.utility_name_eia and l.street_address = r.street_address"
training_session_fname_sname = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.utility_name_eia = r.utility_name_eia and l.street_address = r.street_address

Parameter estimates will be made for the following comparison(s):
    - city
    - address_2
    - zip_code

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - utility_name_eia
    - street_address


Level levenshtein <= 3 on comparison city not observed in dataset, unable to train m value

Level Exact match on comparison address_2 not observed in dataset, unable to train m value

Level levenshtein <= 2 on comparison address_2 not observed in dataset, unable to train m value
Iteration 1: Largest change in params was -0.0904 in the m_probability of address_2, level `Exact match`

Level levenshtein <= 3 on comparison city not observed in dataset, unable to train m value

Level Exact match on comparison address_2 not observed in dataset, unabl

In [274]:
training_blocking_rule = "l.street_address = r.street_address and l.address_2 = r.address_2"
training_session_fname_sname = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.street_address = r.street_address and l.address_2 = r.address_2

Parameter estimates will be made for the following comparison(s):
    - utility_name_eia
    - city
    - zip_code

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - street_address
    - address_2


Level Exact match on comparison utility_name_eia not observed in dataset, unable to train m value

Level levenshtein <= 1 on comparison city not observed in dataset, unable to train m value

Level levenshtein <= 3 on comparison city not observed in dataset, unable to train m value

Level All other comparisons on comparison city not observed in dataset, unable to train m value
Iteration 1: Largest change in params was 0.21 in the m_probability of zip_code, level `All other comparisons`

Level Exact match on comparison utility_name_eia not observed in dataset,

In [278]:
training_blocking_rule = "l.utility_name_eia = r.utility_name_eia"
training_session_fname_sname = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.utility_name_eia = r.utility_name_eia

Parameter estimates will be made for the following comparison(s):
    - city
    - street_address
    - address_2
    - zip_code

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - utility_name_eia

Iteration 1: Largest change in params was 0.316 in probability_two_random_records_match
Iteration 2: Largest change in params was 0.0708 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.0146 in probability_two_random_records_match
Iteration 4: Largest change in params was 0.00318 in probability_two_random_records_match
Iteration 5: Largest change in params was 0.00116 in probability_two_random_records_match
Iteration 6: Largest change in params was 0.00145 in the m_probability of city, level `All other comparisons`
Iteration 7: Largest change in para

Seems like another the city comparison could be changed to have levels 1, 3, all other. Might make the all other less negative.

Address and zip code are probably very dependent, maybe don't include zip code?

In [279]:
linker.match_weights_chart()

Need more emphasis on utility name here? and less on city?

In [280]:
linker.m_u_parameters_chart()

In [281]:
linker.unlinkables_chart()

### Make Predictions

In [282]:
df_predictions = linker.predict()

In [283]:
df_predictions.as_pandas_dataframe().sort_values(by="match_probability", ascending=False)

,match_weight,match_probability,source_dataset_l,unique_id_l,source_dataset_r,unique_id_r,utility_name_eia_l,utility_name_eia_r,gamma_utility_name_eia,bf_utility_name_eia,city_l,city_r,gamma_city,tf_city_l,tf_city_r,bf_city,bf_tf_adj_city,street_address_l,street_address_r,gamma_street_address,bf_street_address,address_2_l,address_2_r,gamma_address_2,bf_address_2,zip_code_l,zip_code_r,gamma_zip_code,bf_zip_code,match_key
56044,30.232275,1.000000e+00,_a,5154,_b,93830,beacon power corporation,beacon power corporation,3,1017.404734,tyngsboro,tyngsboro,3,0.000156,0.000156,129.411544,48.592005,65 middlesex road,65 middlesex road,3,2410.387981,NaN,NaN,-1,1.000000,01879,01879,1,327.782136,0
19219,30.232275,1.000000e+00,_a,7799,_b,53728,selkirk cogen partners limited partnership,selkirk cogen partners limited partnership,3,1017.404734,selkirk,selkirk,3,0.000156,0.000156,129.411544,48.592005,24 power park drive,24 power park drive,3,2410.387981,NaN,NaN,-1,1.000000,12158,12158,1,327.782136,0
89683,30.232275,1.000000e+00,_a,7799,_b,53739,selkirk cogen partners limited partnership,selkirk cogen partners limited partnership,3,1017.404734,selkirk,selkirk,3,0.000156,0.000156,129.411544,48.592005,24 power park drive,24 power park drive,3,2410.387981,NaN,24 power park drive,-1,1.000000,12158,12158,1,327.782136,0
87615,29.910347,1.000000e+00,_a,5119,_b,83372,eme homer city generation limited partnership,eme homer city generation limited partnership,3,1017.404734,homer city,homer city,3,0.000195,0.000195,129.411544,38.873604,1750 power plant road,1750 power plant road,3,2410.387981,NaN,1750 power plant road,-1,1.000000,15748,15748,1,327.782136,0
79463,29.910347,1.000000e+00,_a,1944,_b,24803,green mountain power corporation,green mountain power corporation,3,1017.404734,colchester,colchester,3,0.000195,0.000195,129.411544,38.873604,163 acorn lane,163 acorn lane,3,2410.387981,NaN,163 acorn lane,-1,1.000000,05446,05446,1,327.782136,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894050,-30.634708,5.998382e-10,_a,3573,_b,41783,clean harbors incorporated,new martinsville city of,0,0.701214,norwell,new martinsville,0,0.000039,0.000078,0.000020,1.000000,42 longwater drive,p o drawer 98,0,0.543533,p o box 9149,p o drawer 98,0,0.792006,02061-9149,26155,0,0.397636,4
894051,-30.634708,5.998382e-10,_a,3573,_b,89354,clean harbors incorporated,bioenergy power limited liability company,0,0.701214,norwell,lafayette,0,0.000039,0.001641,0.000020,1.000000,42 longwater drive,p o box 280,0,0.543533,p o box 9149,p o box 280,0,0.792006,02061-9149,47902,0,0.397636,4
735133,-30.634708,5.998382e-10,_a,5900,_b,46513,southern star central corporation,parowan city corporation,0,0.701214,owensboro,parowan,0,0.000430,0.000039,0.000020,1.000000,p o box 20010,p o box 576,0,0.543533,4700 ky highway 56,p o box 576,0,0.792006,42301,84761,0,0.397636,3
894054,-30.634708,5.998382e-10,_a,3573,_b,41904,clean harbors incorporated,new smyrna beach city of,0,0.701214,norwell,new smyrna beach,0,0.000039,0.000195,0.000020,1.000000,42 longwater drive,p o box 100,0,0.543533,p o box 9149,p o box 100,0,0.792006,02061-9149,32170,0,0.397636,4


In [284]:
df_preds = df_predictions.as_pandas_dataframe()

In [285]:
df_preds[df_preds.match_probability > .5].sort_values(by="match_probability")

,match_weight,match_probability,source_dataset_l,unique_id_l,source_dataset_r,unique_id_r,utility_name_eia_l,utility_name_eia_r,gamma_utility_name_eia,bf_utility_name_eia,city_l,city_r,gamma_city,tf_city_l,tf_city_r,bf_city,bf_tf_adj_city,street_address_l,street_address_r,gamma_street_address,bf_street_address,address_2_l,address_2_r,gamma_address_2,bf_address_2,zip_code_l,zip_code_r,gamma_zip_code,bf_zip_code,match_key
538910,0.024773,0.504293,_a,6042,_b,75647,ormat technologies incorporated,ormat nevada incorporated,1,1.691424,reno,NaN,-1,0.002774,NaN,1.000000,1.000000,6225 neil road suite 300,6225 neil road suite 300,3,2410.387981,NaN,6225 neil road suite 300,-1,1.0,89511-1136,NaN,-1,1.000000,2
270329,0.054541,0.509450,_a,7912,_b,109341,questar regulated services company,green beanworks c limited liability company,1,1.691424,salt lake city,salt lake city,3,0.003673,0.003673,129.411544,2.067745,180 east first south street,2180 south 1300 east suite 60,1,23.125552,po box 45360,NaN,-1,1.0,84145,84106,0,0.397636,1
244446,0.054541,0.509450,_a,3323,_b,112066,questar pipeline company,richmond spider solar limited liability company,1,1.691424,salt lake city,salt lake city,3,0.003673,0.003673,129.411544,2.067745,180 east 100 south street,2180 south 1300 east,1,23.125552,p o box 45360,NaN,-1,1.0,84145,84106,0,0.397636,1
244445,0.054541,0.509450,_a,3323,_b,112070,questar pipeline company,pleinmont solar 1 limited liability company,1,1.691424,salt lake city,salt lake city,3,0.003673,0.003673,129.411544,2.067745,180 east 100 south street,2180 south 1300 east,1,23.125552,p o box 45360,NaN,-1,1.0,84145,84106,0,0.397636,1
244443,0.054541,0.509450,_a,3323,_b,112072,questar pipeline company,pleinmont solar 2 limited liability company,1,1.691424,salt lake city,salt lake city,3,0.003673,0.003673,129.411544,2.067745,180 east 100 south street,2180 south 1300 east,1,23.125552,p o box 45360,NaN,-1,1.0,84145,84106,0,0.397636,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95922,29.910347,1.000000,_a,5119,_b,18447,eme homer city generation limited partnership,eme homer city generation limited partnership,3,1017.404734,homer city,homer city,3,0.000195,0.000195,129.411544,38.873604,1750 power plant road,1750 power plant road,3,2410.387981,NaN,NaN,-1,1.0,15748,15748,1,327.782136,0
7285,29.910347,1.000000,_a,1944,_b,24792,green mountain power corporation,green mountain power corporation,3,1017.404734,colchester,colchester,3,0.000195,0.000195,129.411544,38.873604,163 acorn lane,163 acorn lane,3,2410.387981,NaN,NaN,-1,1.0,05446,05446,1,327.782136,0
89683,30.232275,1.000000,_a,7799,_b,53739,selkirk cogen partners limited partnership,selkirk cogen partners limited partnership,3,1017.404734,selkirk,selkirk,3,0.000156,0.000156,129.411544,48.592005,24 power park drive,24 power park drive,3,2410.387981,NaN,24 power park drive,-1,1.0,12158,12158,1,327.782136,0
19219,30.232275,1.000000,_a,7799,_b,53728,selkirk cogen partners limited partnership,selkirk cogen partners limited partnership,3,1017.404734,selkirk,selkirk,3,0.000156,0.000156,129.411544,48.592005,24 power park drive,24 power park drive,3,2410.387981,NaN,NaN,-1,1.0,12158,12158,1,327.782136,0


In [286]:
df_preds.to_pickle("sec_eia_preds_0.pkl")

# Take a look at predictions

Left is the SEC dataframe, right is the EIA dataframe

In [287]:
sec_preds = df_preds.sort_values(by="match_probability", ascending=False).groupby("unique_id_l").first()

In [288]:
eia_preds = df_preds.sort_values(by="match_probability", ascending=False).groupby("unique_id_r").first()

In [294]:
len(sec_preds[sec_preds.match_probability >= .5])/len(sec_df)

0.6908713692946058

In [295]:
len(eia_preds[eia_preds.match_probability >= .5])/len(eia_df)

0.18463339027013348

In [3]:
df_preds = pd.read_pickle("sec_eia_preds_0.pkl")

In [7]:
comp_cols = ["match_probability",
             "utility_name_eia_l",
             "utility_name_eia_r",
             "city_l",
             "city_r",
             "street_address_l",
             "street_address_r",
             "address_2_l",
             "address_2_r",
             "zip_code_l",
             "zip_code_r"]

In [9]:
df_preds[df_preds.match_probability >= .7].sort_values(by="match_probability")[comp_cols].sample(10)

,match_probability,utility_name_eia_l,utility_name_eia_r,city_l,city_r,street_address_l,street_address_r,address_2_l,address_2_r,zip_code_l,zip_code_r
43031,0.792573,fpl group incorporated,fpl energy upton wind limited partnership,juno beach,juno beach,700 universe boulevard,4322084568 cell700 universe bl,NaN,NaN,33408,33408
13998,0.999989,nrg yield limited liability company,bullock road solar 1 limited liability company,princeton,princeton,804 carnegie center,804 carnegie center,NaN,NaN,08540,08540
13675,0.999989,genon mid atlantic limited liability company,bashaw solar 1 limited liability company,princeton,princeton,804 carnegie center,804 carnegie center,NaN,NaN,08540,08540
120224,0.999087,midcoast energy partners limited partnership,manchief power co limited liability company,houston,houston,1100 louisiana street,1001 lousiana street,suite 3300,c o el paso merchant energy,77002,77002
47557,0.810228,biomedical waste systems incorporated,28 livermore hill road solar limited liability...,boston,boston,200 high street,101 summer street 2nd floor,NaN,NaN,02110,02110
140517,0.918422,berrians i gas turbine power limited liability...,minnesota mun pwr agy,minneapolis,minneapolis,901 marquette avenue,120 south sixth street suite 850,NaN,NaN,55402,55402
337740,0.920100,empire district electric trust ii,empire district electric company,joplin,joplin,602 joplin street,602 south joplin street,c o empire district electric company,602 south joplin street,64801,NaN
137789,0.997917,nrg northeast affiliate services incorporated,neo freehold gen limited liability company,minneapolis,minneapolis,901 marquette avenue suite2300,901 marquette avenue,NaN,suite 2300,55402,55402
37121,0.775353,alternus energy incorporated,techren solar iv limited liability company,new york,new york,one world trade center,4 world trade center 51st flo,suite 8500,NaN,10007,10007
41351,0.998709,dcp midstream operating limited partnership,pg solar limited liability company,denver,denver,370 17th street suite 2775,475 17th street suite 570,NaN,NaN,80202,80202
